<a href="https://colab.research.google.com/github/Santosh-Gupta/SparseTorch/blob/master/TruePytorchResearch2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy
import random 
import hashlib

import torch.optim as optim
from tqdm import tqdm
import sys
import tensorflow as tf
import collections
import numpy as np

from collections import deque
numpy.random.seed(12345)

use_cuda = True
# if use_cuda and torch.cuda.is_available():
#     net.cuda()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import hashlib
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile

data_index = 0


In [0]:
import math
import numpy as np
import random
import zipfile
import os
import pandas as pd
import pickle

from numpy import genfromtxt

vocabulary_size = 1666577 

In [0]:
import os
from tempfile import gettempdir
import urllib.request
from six.moves import xrange
import zipfile


In [0]:
#link to hold dictionary folder https://drive.google.com/drive/u/0/folders/1q2D8xlzyQmUjggLStsfVEtgq3_GWxcKl
#in san gupta ml

import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

download_file_from_google_drive('1RHVwT1slwbhPlNTTF1JKS7agc4hQSm5c', 'Data.npy')

In [5]:
my_data = np.load('Data.npy', allow_pickle=True)
print(my_data[0:15])

[[0 list([])]
 [1
  list([421089, 510776, 6403, 1554618, 1451018, 1448326, 1411539, 734702, 263668, 276186, 374145, 712335, 1540518, 732154, 1256014, 370711])]
 [2 list([])]
 [3
  list([896236, 552833, 290985, 744122, 660888, 1492583, 324439, 1497464, 906952, 890270, 800459, 656974, 464637, 432398, 672494, 1501784, 1551199, 169770, 880915, 1257202, 1647789, 431318, 167368, 1309706, 645636, 1589247, 952101, 1594224, 566783, 1020670, 1530466, 572983, 393055, 923629, 1349376, 455838, 168364, 1419708, 670762, 64953])]
 [4
  list([377701, 646875, 1527223, 458740, 1022675, 668690, 910689, 951671, 717587, 1655779, 670477, 66465, 374116, 450320, 83567, 863721, 1328431, 1585189, 1439964])]
 [5
  list([1394328, 658435, 1338541, 1024419, 1193128, 1416126, 600891, 1133836, 1502110, 38954, 200361, 1271103, 914246, 580300, 337729, 316423, 1631441, 75283, 153695, 294419, 904711, 234803, 341096, 350848, 344889, 146171, 610828, 475984, 462863, 768574, 1060750, 753854, 355396, 457861, 1159063, 1074007, 

In [0]:
data_index = 0
epoch_index = 0
recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

def generate_batch(batch_size, inputCount, negRate): #batch size = number of labels
  #inputCount = number of inputs per label
    global data_index, epoch_index
    
    batch = np.ndarray(shape=(batch_size, inputCount), dtype=np.int32) 
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    negs = np.ndarray(shape=(batch_size, negRate), dtype=np.int32)
    
    n=0
    while n < batch_size:
      if len(    set(my_data[data_index, 1])   ) >= inputCount:
        labels[n,0] = my_data[data_index, 0]
        batch[n] = random.sample( set(my_data[data_index, 1]),  inputCount) #gets inputCount number of random inputs
        # print('labels[n,0] ',  labels[n,0])
        # print('batch[n] ',  batch[n])
        # print('labels[n] ',  labels[n])
        excludez = np.concatenate([labels[n] , batch[n]])
        # negs[n] = random.sample([i for i in range(0, vocabulary_size) if i not in excludez ], 64)
        ij=0
        while ij < negRate:
            i = int(random.random() * vocabulary_size)
            if i in excludez or i in negs[n]:
                continue
            negs[n, ij] = i
            ij = ij + 1

        n = n+1
        data_index = (data_index + 1) % len(my_data) #may have to do something like len my_data[:]
        if data_index == 0:
          epoch_index = epoch_index + 1
          print('Completed %d Epochs' % epoch_index)
      else:
        data_index = (data_index + 1) % len(my_data)
        if data_index == 0:
          epoch_index = epoch_index + 1
          print('Completed %d Epochs' % epoch_index)

    return batch, labels, negs     


In [85]:
here, goes, nothing = generate_batch(5, 4, 16) # to do next, insert %len(headernumber)
print('batch', here)
print('labels', goes)
print('negs', nothing)

batch [[ 710144  525974 1118490 1172566]
 [ 376133 1302964  990165   87552]
 [1620250  937582   89573  790989]
 [1021864 1211840 1278123 1509001]
 [1407057  549764 1578486 1205414]]
labels [[30]
 [31]
 [32]
 [33]
 [34]]
negs [[ 608178  849556  106912  262859  293764  967239 1568186  420083 1019243
  1570086  462324  643319  973638  934154  964726  132631]
 [ 741176  483672  660059 1153338  204686  163595  871480  720891  684114
   138528   81496 1297941   86847 1202418   94355  645307]
 [ 671836  478456 1554621 1534952  527760  229759   67100  256847  416427
   198374  613843 1257918  678279  461780  472337   31240]
 [1229705 1653621 1291675 1400285  959024  524165  521770 1133146 1348229
   180810  206058 1258395 1469423  474814 1024907 1322163]
 [1544897  163846  730917 1206408  789527 1454806  732438  593214  804977
  1599056  497777 1102774  310016   72580 1531724 1472930]]


In [0]:

class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        # self.v_embeddings.weight.data.uniform_(-initrange, initrange)

    def forward(self, pos_u, pos_v, neg_v):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        score = torch.mul(emb_u, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(score)
        neg_emb_v = self.v_embeddings(neg_v)
        neg_score = torch.bmm(neg_emb_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)
        return -1 * (torch.sum(score)+torch.sum(neg_score))

    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.
        As this class only record word id, so the map from id to word has to be transfered from outside.
        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


In [0]:

output_file_name = 'theOutpule.file'
emb_size = 1666577
print('embed size is ', emb_size)
emb_dimension = 32
batch_size = 128
window_size = 4
iterationsMax = 10000
initial_lr = 0.025
skip_gram_model = SkipGramModel(emb_size, emb_dimension)
print('got skip gram model')
use_cuda = torch.cuda.is_available()
print('use cuda ? ', use_cuda)
if use_cuda:
    skip_gram_model.cuda()
print('mounted skip gram to cude')
optimizer = optim.Adagrad(
    skip_gram_model.parameters())
print('got optimizer')

for i in range(iterationsMax):

    batch, labels, negz = generate_batch(batch_size=batch_size, num_skips=2, skip_window=1, negRate= 64)
    batchTensor = torch.from_numpy(batch)
    labels = labels.flatten()
    LabelTensor = torch.from_numpy(labels)
    negTensor = torch.from_numpy(negz)

    pos_u = Variable(torch.LongTensor(batchTensor.long()))
    pos_v = Variable(torch.LongTensor(LabelTensor.long()))
    neg_v = Variable(torch.LongTensor(negTensor.long()))

    if use_cuda:
        pos_u = pos_u.cuda()
        pos_v = pos_v.cuda()
        neg_v = neg_v.cuda()

    # optimizer.zero_grad() #set gradients to zero, need to do this at every step or gradients accumulate
    optimizer.zero_grad()

    loss = skip_gram_model.forward(pos_u, pos_v, neg_v)
    runningLoss = runningLoss + loss.data.item()
    # if i%100 == 0:
    #     print('loss is ', loss.data)

    loss.backward()
    optimizer.step()


    if i%500 == 0:
        print('i is ', i)
        print('loss is ', runningLoss/500)
        runningLoss = 0


In [0]:
embedNorm = torch.norm(skip_gram_model.u_embeddings.weight.data, p=2)

In [0]:
normalizedWeights = torch.div(skip_gram_model.u_embeddings.weight.data, embedNorm)

In [0]:
validexamples = Variable(torch.LongTensor([ 4, 23, 45, 28, 29, 12, 11, 16, 20, 24, 28, 32, 34, 82, 94, 39, 54, 21])) 

In [0]:
 validExampleTensors = skip_gram_model.u_embeddings( validexamples.cuda() )

In [0]:
normalizedValid = torch.div(validExampleTensors, embedNorm)

In [0]:
finalSim = torch.mm(normalizedValid, normalizedWeights.t() )

In [0]:
simNumpy = finalSim.cpu().detach().numpy() 

In [0]:
valid_size = [ 4, 23, 45, 283, 292, 12, 113, 167, 200, 204, 208, 432, 354, 382, 194, 339, 54, 221]

for i in range(len(valid_size)):
    valid_word = reverse_dictionary[valid_size[i]]
    top_k = 80  # number of nearest neighbors
    nearest = (-simNumpy[i, :]).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in xrange(top_k):
        close_word = reverse_dictionary[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)